In [2]:
from keras.layers import Dense, LSTM, GRU, Dropout, Flatten, Embedding,  BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import keras
from keras.models import Model,Sequential
from keras.utils import to_categorical
from keras import regularizers
from keras.optimizers import SGD
from clr_callback import *
%matplotlib inline
import time
import pickle
import os
import math
import torch.nn as nn
from torch.nn.modules.utils import _triple
from keras.utils import to_categorical
import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report



In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
SPLIT_METHOD_child = "CHILD"
SPLIT_METHOD_session = "SESSION"
SPLIT_METHOD_random = "RANDOM"

# WITH FACE, BODY, HANDS (411) =>
FEAT_NUM_all = 201

# WITH BODY AND HANDS (201) =>
FEAT_NUM_body = (25 + 21 + 21) * 3

# WITH FACE ONLY (210) =>
FEAT_NUM_face = 70 * 3

SEQ_LENGTH = 1

NB_CLASS = 6
CSV_FILE = "ASD_36_res1.csv"
VARS_DIR = "var"


In [5]:
import pandas
data = pandas.read_csv('ASD_36_res1.csv')

In [6]:
X = data.iloc[:, 3:78]
y = data['engagement']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=1)

In [7]:
X_train.shape


(253124, 75)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [9]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(227811, 75)
(25313, 75)
(41207, 75)


In [10]:
X_train = X_train.to_numpy()
X_test =X_test.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_val = y_val.to_numpy()

In [11]:
type(X_train)

numpy.ndarray

In [12]:

early_stop = EarlyStopping(monitor='loss', patience=2)


In [13]:
xx = tf.convert_to_tensor(X_train.reshape(227811,1 , 75))
xv = tf.convert_to_tensor(X_val.reshape(25313, 1, 75))
xt = tf.convert_to_tensor(X_test.reshape(41207,1, 75))
yy = tf.convert_to_tensor(y_train.reshape(227811, 1))
yv = tf.convert_to_tensor(y_val.reshape(25313, 1))
yt = tf.convert_to_tensor(y_test.reshape(41207, 1))

In [14]:
yy= to_categorical(yy)
yv =  to_categorical(yv)
yt = to_categorical(yt)

In [15]:
model = keras.Sequential()
model.add(LSTM(32, input_shape=(1, 75), activation =tf.keras.layers.LeakyReLU(alpha=0.1), dropout=0.2, recurrent_dropout = 0.2, return_sequences=True))

model.add(LSTM(32, activation = tf.keras.layers.LeakyReLU(alpha=0.1), dropout=0.2, recurrent_dropout = 0.2))

model.add(keras.layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))

model.add(keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))

model.add(keras.layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))

model.add(keras.layers.Dense(256, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))

model.add(keras.layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))

model.add(keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))

model.add(keras.layers.Dense(16, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(keras.layers.Dense(6, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 32)             13824     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               3

In [16]:
from keras.optimizers import SGD
opt = "adam"#SGD(lr=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [17]:
hist = model.fit(x = xx , y = yy, epochs=8, validation_data =(xv, yv), batch_size= 32)


Epoch 1/8
7120/7120 [==============================] - 69s 10ms/step - loss: 1.5179 - accuracy: 0.4296 - val_loss: 1.4679 - val_accuracy: 0.4454
Epoch 2/8
7120/7120 [==============================] - 68s 10ms/step - loss: 1.4712 - accuracy: 0.4461 - val_loss: 1.4721 - val_accuracy: 0.4406
Epoch 3/8
7120/7120 [==============================] - 71s 10ms/step - loss: 1.4605 - accuracy: 0.4468 - val_loss: 1.4514 - val_accuracy: 0.4452
Epoch 4/8
7120/7120 [==============================] - 70s 10ms/step - loss: 1.4528 - accuracy: 0.4482 - val_loss: 1.4490 - val_accuracy: 0.4472
Epoch 5/8
7120/7120 [==============================] - 70s 10ms/step - loss: 1.4504 - accuracy: 0.4487 - val_loss: 1.4480 - val_accuracy: 0.4463
Epoch 6/8
7120/7120 [==============================] - 73s 10ms/step - loss: 1.4469 - accuracy: 0.4499 - val_loss: 1.4441 - val_accuracy: 0.4440
Epoch 7/8
7120/7120 [==============================] - 71s 10ms/step - loss: 1.4451 - accuracy: 0.4504 - val_loss: 1.4344 - val_ac

In [18]:
h = model.evaluate(xt,yt)

1288/1288 [==============================] - 2s 2ms/step - loss: 1.4181 - accuracy: 0.4585
